In [2]:
!ls ../data/house-prices/

processed_df.parquet
test.csv
train.csv


In [3]:
import pandas as pd


# Read data

In [36]:
DATA_PATH = '../data/house-prices/'
DATASET_PATH = DATA_PATH + 'train.csv'
label_col = 'SalePrice'

In [63]:
df_master = pd.read_csv(DATASET_PATH)
df = df_master.copy()
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [64]:
df[useful_features].duplicated(keep='first').sum()

32

In [65]:
df = df[~df[useful_features].duplicated(keep='first')]
# or just use drop_duplicates
df.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [66]:
df = df.reset_index(drop=True)
df.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1423,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1424,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1425,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1426,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1427,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [67]:
X, y = df.drop(columns=[label_col]), df[label_col]

In [68]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [69]:
y_train

266     148000
327     119000
348      95000
478     155000
59      124900
         ...  
1095    188000
1130    116900
1294    275000
860     200500
1126     80000
Name: SalePrice, Length: 956, dtype: int64

### Filter useless columns

In [6]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [70]:
useful_features = ['Foundation', 'KitchenQual', 'TotRmsAbvGrd', 'WoodDeckSF', 'YrSold', '1stFlrSF']
X_train = X_train[useful_features]
X_train

,Foundation,KitchenQual,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF
266,CBlock,TA,6,370,2007,1113
327,CBlock,TA,10,155,2007,1728
348,CBlock,TA,5,48,2008,616
478,PConc,Ex,7,0,2009,916
59,CBlock,TA,4,196,2008,780
...,...,...,...,...,...,...
1095,PConc,TA,8,224,2008,773
1130,PConc,TA,5,0,2008,572
1294,PConc,Gd,7,198,2006,1787
860,PConc,Gd,7,0,2010,804


In [71]:
X_test = X_test[useful_features]
X_test

,Foundation,KitchenQual,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF
123,PConc,Gd,5,0,2008,1199
1189,PConc,Gd,8,635,2010,1055
462,CBlock,TA,5,0,2007,1277
351,PConc,Gd,6,0,2007,1200
1009,CBlock,TA,6,0,2007,912
...,...,...,...,...,...,...
285,CBlock,TA,5,0,2010,900
1314,PConc,Gd,8,146,2006,920
1352,BrkTil,TA,5,0,2010,864
543,CBlock,TA,5,104,2007,768


### Impute missing values

In [47]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 978 entries, 615 to 1126
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Foundation    978 non-null    object
 1   KitchenQual   978 non-null    object
 2   TotRmsAbvGrd  978 non-null    int64 
 3   WoodDeckSF    978 non-null    int64 
 4   YrSold        978 non-null    int64 
 5   1stFlrSF      978 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 53.5+ KB


In [72]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 472 entries, 123 to 818
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Foundation    472 non-null    object
 1   KitchenQual   472 non-null    object
 2   TotRmsAbvGrd  472 non-null    int64 
 3   WoodDeckSF    472 non-null    int64 
 4   YrSold        472 non-null    int64 
 5   1stFlrSF      472 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 25.8+ KB


No missing values

### Check duplicates

In [9]:
help(df[useful_features].duplicated)

Help on method duplicated in module pandas.core.frame:

duplicated(subset: 'Hashable | Sequence[Hashable] | None' = None, keep: 'DropKeep' = 'first') -> 'Series' method of pandas.core.frame.DataFrame instance
    Return boolean Series denoting duplicate rows.
    
    Considering certain columns is optional.
    
    Parameters
    ----------
    subset : column label or sequence of labels, optional
        Only consider certain columns for identifying duplicates, by
        default use all of the columns.
    keep : {'first', 'last', False}, default 'first'
        Determines which duplicates (if any) to mark.
    
        - ``first`` : Mark duplicates as ``True`` except for the first occurrence.
        - ``last`` : Mark duplicates as ``True`` except for the last occurrence.
        - False : Mark all duplicates as ``True``.
    
    Returns
    -------
    Series
        Boolean series for each duplicated rows.
    
    See Also
    --------
    Index.duplicated : Equivalent method 

In [62]:
X_test.duplicated(keep='first').sum()

5

- Remove all the duplicates except one

- Some analysis, skip it if you did not understand it ;)

In [13]:
df_tmp = df_master[useful_features+[label_col]]
df_tmp[df_tmp.duplicated(keep=False)]

,Foundation,KitchenQual,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF,SalePrice
145,PConc,Gd,7,0,2006,970,130000
193,PConc,Gd,7,0,2006,970,130000


In [14]:
df_tmp = df_master[useful_features+[label_col]]
df_tmp[df_tmp[useful_features].duplicated(keep=False)]

,Foundation,KitchenQual,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF,SalePrice
18,PConc,Gd,6,0,2008,1114,159000
76,CBlock,TA,4,0,2008,952,135750
87,PConc,Gd,4,0,2009,612,164500
89,PConc,TA,5,0,2007,990,123600
102,Slab,TA,8,0,2009,1535,118964
126,CBlock,TA,5,0,2007,958,128000
145,PConc,Gd,7,0,2006,970,130000
193,PConc,Gd,7,0,2006,970,130000
194,CBlock,TA,5,0,2008,864,127000
203,PConc,Gd,3,149,2008,848,149000


In [15]:
df_tmp2 = df_tmp[df_tmp[useful_features].duplicated(keep=False)]
df_tmp2[(df_tmp2['Foundation'] == 'PConc') & (df_tmp2['KitchenQual'] == 'Gd') & (df_tmp2['TotRmsAbvGrd'] == 6)]

,Foundation,KitchenQual,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF,SalePrice
18,PConc,Gd,6,0,2008,1114,159000
282,PConc,Gd,6,172,2009,1314,207500
511,PConc,Gd,6,132,2006,1368,202665
1157,PConc,Gd,6,172,2009,1314,230000
1306,PConc,Gd,6,132,2006,1368,202500
1341,PConc,Gd,6,0,2008,1114,155000


### Scale continuous feautres

In [74]:
continuous_columns = X_train[useful_features].select_dtypes(include='number').columns
continuous_columns

Index(['TotRmsAbvGrd', 'WoodDeckSF', 'YrSold', '1stFlrSF'], dtype='object')

In [77]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train[continuous_columns])

StandardScaler()

In [78]:
scaled_columns_Xtrain = scaler.transform(X_train[continuous_columns])
scaled_columns_Xtrain

array([[-0.37686857,  2.17600714, -0.62349243, -0.1457359 ],
       [ 2.074059  ,  0.46789767, -0.62349243,  1.40975046],
       [-0.98960046, -0.38218472,  0.13775759, -1.40277122],
       ...,
       [ 0.23586332,  0.80951957, -1.38474244,  1.55897598],
       [ 0.23586332, -0.76353009,  1.66025761, -0.92727295],
       [-0.98960046, -0.76353009,  1.66025761, -1.26113344]])

In [79]:
continuous_features_train_df = pd.DataFrame(data=scaled_columns_Xtrain, columns=continuous_columns)
continuous_features_train_df.head()

,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF
0,-0.376869,2.176007,-0.623492,-0.145736
1,2.074059,0.467898,-0.623492,1.409750
2,-0.989600,-0.382185,0.137758,-1.402771
3,0.235863,-0.763530,0.899008,-0.643997
4,-1.602332,0.793630,0.137758,-0.987975


In [80]:
scaled_columns_Xtest = scaler.transform(X_test[continuous_columns])
scaled_columns_Xtest

array([[-0.98960046, -0.76353009,  0.13775759,  0.07177926],
       [ 0.84859522,  4.28135138,  1.66025761, -0.29243218],
       [-0.98960046, -0.76353009, -0.62349243,  0.26906046],
       ...,
       [-0.98960046, -0.76353009,  1.66025761, -0.77551819],
       [-0.98960046,  0.06271822, -0.62349243, -1.01832581],
       [-0.98960046, -0.76353009,  0.13775759, -0.97785787]])

In [81]:
continuous_features_test_df = pd.DataFrame(data=scaled_columns_Xtest, columns=continuous_columns)
continuous_features_test_df.head()

,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF
0,-0.989600,-0.763530,0.137758,0.071779
1,0.848595,4.281351,1.660258,-0.292432
2,-0.989600,-0.763530,-0.623492,0.269060
3,-0.376869,-0.763530,-0.623492,0.074309
4,-0.376869,-0.763530,-0.623492,-0.654114


### Categorical features

In [20]:
df[useful_features].dtypes

Foundation      object
KitchenQual     object
TotRmsAbvGrd     int64
WoodDeckSF       int64
YrSold           int64
1stFlrSF         int64
dtype: object

In [83]:
categorical_columns = df[useful_features].select_dtypes(include='object').columns
categorical_columns

Index(['Foundation', 'KitchenQual'], dtype='object')

In [22]:
categorical_features_df = pd.get_dummies(df[categorical_columns],dtype=int)
categorical_features_df.head()

,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA
0,0,0,1,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,1,0


In [84]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoder.fit(X_train[categorical_columns])

OneHotEncoder()

In [87]:
encoded_data = encoder.transform(X_train[categorical_columns])
categorical_features_train_df = pd.DataFrame(data = encoded_data, columns=categorical_columns)
categorical_features_train_df

ValueError: Shape of passed values is (956, 1), indices imply (956, 2)

In [89]:
encoded_data

<956x10 sparse matrix of type '<class 'numpy.float64'>'
	with 1912 stored elements in Compressed Sparse Row format>

- Join dataframes

In [23]:
final_df = continuous_features_df.join(categorical_features_df).join(df[label_col])
final_df.head()

,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,SalePrice
0,0.912210,-0.752176,0.138777,-0.793434,0,0,1,0,0,0,0,0,1,0,208500
1,-0.318683,1.626195,-0.614439,0.257140,0,1,0,0,0,0,0,0,0,1,181500
2,-0.318683,-0.752176,0.138777,-0.627826,0,0,1,0,0,0,0,0,1,0,223500
3,0.296763,-0.752176,-1.367655,-0.521734,1,0,0,0,0,0,0,0,1,0,140000
4,1.527656,0.780197,0.138777,-0.045611,0,0,1,0,0,0,0,0,1,0,250000


# Model training

- Split dataset

In [24]:
X, y = final_df.drop(columns=[label_col]), final_df[label_col]

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [82]:
X_train.head()

,Foundation,KitchenQual,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF
266,CBlock,TA,6,370,2007,1113
327,CBlock,TA,10,155,2007,1728
348,CBlock,TA,5,48,2008,616
478,PConc,Ex,7,0,2009,916
59,CBlock,TA,4,196,2008,780


- Train model

In [26]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [27]:
model.fit(X_train, y_train)

LinearRegression()

# Model evaluation

In [28]:
y_pred = model.predict(X_test)

In [29]:
y_pred[y_pred < 0] = 0

In [30]:
import numpy as np
from sklearn.metrics import mean_squared_log_error

def compute_rmsle(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
    return round(rmsle, precision)

In [31]:
compute_rmsle(y_test, y_pred)

0.22

## Testing dataframe equality

In [34]:
final_df.to_parquet(DATA_PATH + 'processed_df.parquet', index=False)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [31]:
!ls ../../data/house-prices/

processed_df..parquet processed_df.parquet  train.csv
processed_df.csv      test.csv


In [32]:
processed_df = pd.read_parquet(DATA_PATH + 'processed_df.parquet')
processed_df.head()

,TotRmsAbvGrd,WoodDeckSF,YrSold,1stFlrSF,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,SalePrice
0,0.912210,-0.752176,0.138777,-0.793434,0,0,1,0,0,0,0,0,1,0,208500
1,-0.318683,1.626195,-0.614439,0.257140,0,1,0,0,0,0,0,0,0,1,181500
2,-0.318683,-0.752176,0.138777,-0.627826,0,0,1,0,0,0,0,0,1,0,223500
3,0.296763,-0.752176,-1.367655,-0.521734,1,0,0,0,0,0,0,0,1,0,140000
4,1.527656,0.780197,0.138777,-0.045611,0,0,1,0,0,0,0,0,1,0,250000


In [33]:
pd.testing.assert_frame_equal(processed_df, final_df)

In [34]:
pd.testing.assert_frame_equal(processed_df, final_df.drop(columns=[label_col]))

AssertionError: DataFrame are different

DataFrame shape mismatch
[left]:  (1460, 15)
[right]: (1460, 14)